In [57]:
import numpy as np
import csv
import os
import sklearn
import pandas
from pandas import read_csv as read
from sklearn import svm, preprocessing, ensemble
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from scipy.spatial import distance
import heapq

In [3]:
path_train = "lab2data/arcene_train.data"
data_train = read(path_train, delimiter=" ")
data_train.columns = [i for i in range(1, 10001)] + ['class']
data_train = data_train.drop(['class'], axis=1)
data_train.head()

,1,2,3,4,5,6,7,8,9,10,...,9991,9992,9993,9994,9995,9996,9997,9998,9999,10000
0,0,41,82,165,60,554,379,0,71,0,...,213,605,69,7,473,0,57,0,284,423
1,0,0,1,40,0,451,402,0,0,0,...,235,593,28,0,24,0,90,0,34,508
2,0,56,44,275,14,511,470,0,0,0,...,91,600,0,26,86,0,102,0,0,469
3,105,0,141,348,0,268,329,0,0,1,...,813,0,0,0,0,190,301,0,0,354
4,38,62,0,251,75,515,0,9,85,300,...,272,454,0,36,2,369,194,18,59,340


In [4]:
path_valid = "lab2data/arcene_valid.data"
data_valid = read(path_valid, delimiter=" ")
data_valid.columns = [i for i in range(1, 10001)] + ['class']
data_valid = data_valid.drop(['class'], axis=1)
data_valid.head()

,1,2,3,4,5,6,7,8,9,10,...,9991,9992,9993,9994,9995,9996,9997,9998,9999,10000
0,0,7,0,7,0,251,0,0,22,222,...,698,673,0,0,18,201,317,0,34,199
1,0,32,0,470,53,493,0,0,140,147,...,267,461,20,34,165,225,53,0,47,219
2,0,77,0,202,0,432,472,0,60,0,...,181,555,13,0,488,0,71,0,177,487
3,0,34,102,522,2,474,473,0,39,0,...,114,630,0,29,192,0,74,0,5,416
4,89,31,156,334,0,250,227,0,0,0,...,823,0,0,0,0,212,345,0,390,356


In [5]:
path_train_l = "lab2data/arcene_train.labels"
data_train_l = read(path_train_l, delimiter=" ")
path_valid_l = "lab2data/arcene_valid.labels"
data_valid_l = read(path_valid_l, delimiter=" ")

In [6]:
def norma(data):
    data = (data - data.mean()) / data.std()
    return data

print(norma(data_train))

       1         2         3         4         5         6         7      \
0  -0.750561  0.159916  0.362605 -1.023644  1.616537  1.323625  0.796845   
1  -0.750561 -0.927350 -0.815567 -1.788780 -0.654030  0.371455  0.920342   
2  -0.750561  0.557696 -0.190118 -0.350324 -0.124231  0.926117  1.285463   
3   1.354695 -0.927350  1.220780  0.096515 -0.654030 -1.320263  0.528374   
4   0.011341  0.716808 -0.830113 -0.497230  2.184179  0.963095 -1.238169   
5   0.773243  1.194144  2.602587 -0.729832 -0.654030 -0.802579  0.700195   
6   0.191792 -0.821275  2.180772 -0.674742 -0.654030 -0.811823 -0.540143   
7  -0.750561 -0.476533 -0.757386 -1.531694 -0.654030 -0.146229  1.290833   
8   0.011341  2.069261  1.875320  0.757593  2.600450 -1.384974 -0.545512   
9  -0.750561 -0.927350  0.653512 -1.354183 -0.654030  1.314380  0.909603   
10 -0.409710 -0.237865 -0.830113 -0.203418 -0.654030  0.833674 -1.238169   
11 -0.750561  1.220663 -0.219209 -0.937949  0.632624  0.750474  1.161966   
12  1.374745

In [7]:
X_train, X_valid, y_train, y_valid = data_train, data_valid, np.ravel(data_train_l), np.ravel(data_valid_l)

In [102]:
rf = ensemble.RandomForestClassifier(n_estimators=100, random_state=11)
rf.fit(X_train, y_train)
print(rf.score(X_valid, y_valid))

0.7373737373737373


# Filter1 Feature importance

In [115]:
importances = rf.feature_importances_
indices = np.argsort(importances)[::-1]

print("Feature importances:")
for f, idx in enumerate(indices):
    print("{:2d}. feature '{:5d}' ({:.4f})".format(f + 1, X_train.columns[idx], importances[idx]))

Feature importances:
 1. feature '    3' (0.0493)
 2. feature '  107' (0.0242)
 3. feature '  135' (0.0236)
 4. feature '   61' (0.0229)
 5. feature '   16' (0.0217)
 6. feature '   11' (0.0214)
 7. feature '   84' (0.0173)
 8. feature '   15' (0.0171)
 9. feature '   68' (0.0168)
10. feature '   83' (0.0168)
11. feature '   12' (0.0168)
12. feature '    2' (0.0166)
13. feature '   76' (0.0161)
14. feature '  124' (0.0155)
15. feature '   62' (0.0150)
16. feature '   44' (0.0143)
17. feature '  101' (0.0136)
18. feature '  132' (0.0133)
19. feature '   50' (0.0131)
20. feature '   33' (0.0123)
21. feature '   60' (0.0118)
22. feature '    6' (0.0117)
23. feature '  121' (0.0117)
24. feature '   10' (0.0116)
25. feature '  118' (0.0113)
26. feature '   93' (0.0112)
27. feature '   42' (0.0110)
28. feature '   79' (0.0109)
29. feature '   19' (0.0109)
30. feature '  128' (0.0106)
31. feature '   47' (0.0098)
32. feature '   25' (0.0097)
33. feature '    5' (0.0096)
34. feature '   69' (0

In [116]:
best_features = indices[:135]
best_features_names = X_train.columns[best_features]
best_f = [int(i) for i in best_features_names]
print(best_f)

[3, 107, 135, 61, 16, 11, 84, 15, 68, 83, 12, 2, 76, 124, 62, 44, 101, 132, 50, 33, 60, 6, 121, 10, 118, 93, 42, 79, 19, 128, 47, 25, 5, 69, 43, 81, 85, 26, 111, 9, 73, 88, 30, 63, 95, 71, 123, 100, 112, 18, 127, 36, 17, 105, 96, 131, 130, 23, 98, 87, 13, 35, 54, 45, 110, 27, 129, 91, 66, 109, 4, 106, 119, 74, 80, 120, 8, 67, 64, 122, 20, 90, 38, 86, 46, 94, 40, 52, 48, 51, 21, 58, 103, 134, 53, 41, 125, 133, 92, 77, 104, 116, 24, 75, 70, 65, 126, 28, 1, 99, 82, 7, 115, 39, 89, 113, 102, 31, 22, 72, 78, 117, 32, 59, 55, 29, 37, 34, 49, 56, 14, 114, 97, 108, 57]


In [117]:
print(X_train[best_f])
print(len(y_train))

    3    107  135  61   16   11   84   15   68   83  ...   29   37   34   49   \
0    82   86   16    4    0    0    0  364  183   29 ...   564    0   45    8   
1     1  153    0    0    0    0    0  269  103   51 ...   494    0    8    0   
2    44  167   59   38    0    0   79  270  155   20 ...   435    0    0    0   
3   141    0    0  551   17    0    0  312   49  485 ...   428    0   34    0   
4     0  123    0  443   26    0    0  446    0  194 ...   473    0    0   17   
5   236    7    0  358    0    0   49  313  111  466 ...   445    0   31    0   
6   207   37    0  334    0    0   50  293   67  496 ...   435   27   93    0   
7     5  197    0    0    0    0   41  209  113   56 ...   377   50    0    0   
8   186   84    0  555  175  132    6  207   25  141 ...   250    0  106   89   
9   102   75    0    0    0    0   66  377  184   41 ...   592    0  132    0   
10    0   84    0  456   34   98    0  434    0  169 ...   453    0   58   18   
11   42  141   21   30    0 

In [124]:
rf2 = ensemble.RandomForestClassifier(n_estimators=100, random_state=11)
rf2.fit(X_train[best_f], y_train)
print(rf2.score(X_valid[best_f], y_valid))

0.797979797979798


# Filter2 euclid

In [68]:
# normalize the data attributes
normalized_X = preprocessing.normalize(X_train)

rf4 = ensemble.RandomForestClassifier(n_estimators=100, random_state=11)

dst_array = []

for i in range(0, 10000):
    dst_array.append(distance.euclidean(normalized_X[:,i], y_train))
dst_array = np.asarray(dst_array)
ind = heapq.nsmallest(20, range(len(dst_array)), dst_array.take)

print(ind)
print(dst_array[6125], dst_array[6])
rf4.fit(X_train[ind], y_train)
print(rf4.score(X_valid[ind], y_valid))

[6125, 7822, 293, 9446, 912, 9641, 403, 765, 6203, 6, 5183, 1540, 9738, 262, 1811, 7907, 9092, 7111, 8845, 9869]
9.92797698823357 9.928903560892307
0.8080808080808081


# Filter 3 corr coef

In [106]:
rf5 = ensemble.RandomForestClassifier(n_estimators=100, random_state=11)
normalized_X = preprocessing.normalize(X_train)

corr_array = []

for i in range(0, 10000):
    corr_array.append(np.corrcoef(normalized_X[:,i], y_train)[0, 1])
corr_array = np.asarray(corr_array)
#ind1 = heapq.nsmallest(10, range(len(corr_array)), corr_array.take) 
ind2 = heapq.nlargest(45, range(len(corr_array)), corr_array.take)
#print(ind1 + ind2)
rf4.fit(X_train[ind2], y_train)
print(rf4.score(X_valid[ind2], y_valid))

e:\prgfiles\miniconda3\lib\site-packages\numpy\lib\function_base.py:2400: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
e:\prgfiles\miniconda3\lib\site-packages\numpy\lib\function_base.py:2401: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


0.8282828282828283


# Wrapper

In [152]:
import random
rf3 = ensemble.RandomForestClassifier(n_estimators=100, random_state=11)
c = list(range(1, 10001))
c = random.sample(c, 10000)
g = random.sample(c, 10)
result = rf3.fit(X_train[g], y_train)
res=result.score(X_valid[g], y_valid)
print(res)
for i in range(1, 10001):          #Forward Selection wrapper
    if c[i] in g:
        continue
    g.append(c[i])
    print(g)
    result = result.fit(X_train[g], y_train)
    score = result.score(X_valid[g], y_valid)
    print(score)
    if score < res:
        break
    res = score

0.7070707070707071
[8227, 7616, 3420, 467, 9862, 8226, 5251, 3146, 2263, 136, 9314]
0.7171717171717171
[8227, 7616, 3420, 467, 9862, 8226, 5251, 3146, 2263, 136, 9314, 1529]
0.7777777777777778
[8227, 7616, 3420, 467, 9862, 8226, 5251, 3146, 2263, 136, 9314, 1529, 3810]
0.7575757575757576
